In [4]:
# ------------------------------------------------------------
# 1. Importations
# ------------------------------------------------------------
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [5]:
# ------------------------------------------------------------
# 2. Chargement du dataset
# ------------------------------------------------------------
DATASET_PATH = "../data/flower_images"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=None,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)

val_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=None,
    class_mode="categorical",
    shuffle=False,
    subset="validation"
)

print("Classes :", train_gen.class_indices)


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Classes : {'Lilly': 0, 'Lotus': 1, 'Orchid': 2, 'Sunflower': 3, 'Tulip': 4}


In [6]:
# ------------------------------------------------------------
# 3. Chargement du modèle baseline (gelé)
# ------------------------------------------------------------
base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Modèle initial (gelé)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)


In [7]:
# ------------------------------------------------------------
# 4. Dégeler une partie du modèle
# Ici, on dégèle les 50 dernières couches (standard pour InceptionV3).
# ------------------------------------------------------------

# Définir combien de couches on dégèle
N = 50

for layer in base_model.layers[:-N]:
    layer.trainable = False

for layer in base_model.layers[-N:]:
    layer.trainable = True

print("Couches entraînables :", sum([layer.trainable for layer in model.layers]))
print("Couches gelées :", sum([not layer.trainable for layer in model.layers]))


Couches entraînables : 53
Couches gelées : 261


In [8]:
# ------------------------------------------------------------
# 5. Recompiler le modèle
# Important : en fine-tuning, on utilise un learning rate très petit.
# ------------------------------------------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [9]:
# ------------------------------------------------------------
# 6. Callbacks
# ------------------------------------------------------------
callbacks = [
    EarlyStopping(
        patience=5,
        monitor="val_loss",
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=3,
        min_lr=1e-7
    ),
    ModelCheckpoint(
        "../models/inception_finetuned.h5",
        save_best_only=True,
        monitor="val_loss"
    )
]


In [10]:
# ------------------------------------------------------------
# 7. Entraînement du modèle
# ------------------------------------------------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)


C:\Users\pc\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 10/125 ━━━━━━━━━━━━━━━━━━━━ 4:59 3s/step - accuracy: 0.1843 - loss: 1.8724

KeyboardInterrupt: 

In [ ]:
# ------------------------------------------------------------
# 8. Courbes d’apprentissage
# ------------------------------------------------------------
plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train acc")
plt.plot(history.history["val_accuracy"], label="val acc")
plt.title("Accuracy")
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="val loss")
plt.title("Loss")
plt.legend()

plt.show()
